### This program calculates the spaial error and shape error between a reference image and a drawing
- Shape Error: Measured by MSE
- Spatial Error: Rotation, Translation, Scaling

### Basic Setup

#### Load dependencies and define directory hierarchy

In [1]:
import sys
import os

import pymongo as pm ## first establish ssh tunnel to server where database is running
import numpy as np
from numpy import *
import cv2
import pandas as pd
import svg_distance_helpers as rsh

import affine_registration_2d as ar

#### File hierarchy

In [6]:
analysis_dir = os.getcwd()
iteration_name = 'cdm_run_v5'

ref_dir = os.path.join(analysis_dir, 'tracing_ref_400')
data_dir = os.path.join(analysis_dir, 'tracing_{}'.format(iteration_name))

loss = 'ncc'
trans_dir = os.path.join(analysis_dir, 'transformed_{}'.format(loss)) # store transformed images
if not os.path.exists(trans_dir):
    os.mkdir(trans_dir)

canvas_side = 400 # image size
tracing = ['square','shape']

data_output = 'kiddraw_tracing_{}.csv'.format(iteration_name, loss)

#### Check how many items get transformed

In [ ]:
# get data from csv file
data = pd.read_csv(data_output)
try:
    data = data.drop("Unnamed: 0", axis=1)
except:
    print "Already droped"
print data.count()

#### Generate binary reference shapes

In [7]:
really_run_this = 0
ref_path = []

if really_run_this:
    for t in tracing:
        fname = os.path.join(ref_dir, '{}.png'.format(t))
        img_ref = cv2.imread(fname)
        img_ref = rsh.color_to_num(img_ref, threshold=0, white_background=False)
        cv2.imwrite(ofname, 255-img_ref*255)

for t in tracing:
    ofname = os.path.join(ref_dir, '{}_ref.png'.format(t))
    ref_path.append(ofname)

### Calculate Shape Error and Spatial Error

In [8]:
reload(ar)

session_id = []
age = []
category =[] 
pre_tran = []
post_tran = []
rotate = []
translate = []
scale = []
has_ref = []

In [9]:
for ti, t in enumerate(tracing):
    t_dir = os.path.join(data_dir, t)
    
    for d in os.listdir(t_dir)[:2]:
        if not d.endswith('.png'):
            continue

        child_info = d.split('_')
        c_age = int(child_info[0].replace('age',''))
        c_sessionid = '_'.join([child_info[1], child_info[2], child_info[3]])

        img_draw = os.path.join(t_dir, d)

        if os.path.exists(img_draw):
            img_ref = ref_path[ti]
            output_path = os.path.join(trans_dir, '{}_{}_wrap.png'.format(c_sessionid, t))
            init_loss, final_loss, ro, tran, s, warped = ar.affine_reg(img_draw, img_ref, output_path)

            # prepare the dataframe
            age.append(c_age)
            session_id.append(c_sessionid)
            category.append(t.replace('this ', ''))
            pre_tran.append(init_loss)
            post_tran.append(final_loss)
            rotate.append(ro)
            translate.append(tran)
            scale.append(s)
            
            if t.startswith('this'):
                has_ref.append(True)
            else:
                has_ref.append(False)

In [11]:
really_run_this = 1 
## compile data
if really_run_this:
    data = pd.DataFrame([session_id, age, category, pre_tran, post_tran, rotate, translate, scale, has_ref])
    data = data.transpose()
    data.columns = ['session_id','age', 'category', 'pre_tran', 'post_tran', 'rotate', 'translate', 'scale', 'has_ref']
## save out the data
data.to_csv(data_output)